- Link for minsearch: https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
- Link for documents: https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
- Notebook to parse documents: https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/parse-faq.ipynb

## Retrieval

In [14]:
from minsearch import Index
import json

In [15]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [16]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']: 
        doc['course'] = course_dict['course']
        documents.append(doc)

In [17]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [18]:
index = Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [19]:
index.fit(documents)

In [20]:
q = 'the course has already started, can I still enroll?'

In [21]:
boost = {
    'question': 3.0,
    'section': 0.5
    }

index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Generation with Groq (OpenAI alternatives)

In [22]:
from groq import Groq
import os
from dotenv import load_dotenv

In [23]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [24]:
client = Groq(
    api_key=GROQ_API_KEY
)

In [25]:
response = client.chat.completions.create(
    messages = [
        {
            "role":"user",
            "content": q
        }
    ],
    model="mixtral-8x7b-32768"
)

response.choices[0].message.content

"Whether or not you can still enroll in a course that has already started depends on a number of factors, such as the policies of the institution or organization offering the course, the specific course format and structure, and the point in the course at which you are attempting to enroll.\n\nIn some cases, institutions may allow late enrollments in certain courses, particularly if the course is delivered online or in a flexible format. However, this is not always the case, and there may be restrictions on when you can enroll or limitations on the number of late enrollments that are allowed.\n\nIf you are interested in enrolling in a course that has already started, your first step should be to contact the institution or organization offering the course and inquire about their late enrollment policies. They can provide you with more information on whether or not it is possible to enroll at this time, and what steps you would need to take in order to do so.\n\nIt's also worth consideri

## Cleaned RAG flow

In [26]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [27]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [28]:
def llm(prompt):
    response = client.chat.completions.create(
        messages = [
            {
                "role":"user",
                "content": prompt
            }
        ],
        model="mixtral-8x7b-32768"
    )

    return response.choices[0].message.content

In [29]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
query = 'how do I run kafka?'

rag(query)

"To run Apache Kafka, you need to have Kafka installed and running on your machine. However, the FAQ database seems to provide information on how to run Kafka-related projects or resolve errors in Python or Java code, but it doesn't include instructions on how to install and start the actual Kafka broker.\n\nTo install and run Apache Kafka, you can follow the official Apache Kafka documentation: <https://kafka.apache.org/quickstart>\n\nAfter setting up Kafka, you can run the provided Java or Python code in the FAQ database by following the appropriate instructions based on the context. For example, if you want to run a Java producer, you can use the command provided in the first context snippet:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the name of your Java project's JAR file.\n\nSimilarly, for Python-related questions and issues, you can refer to the respective context sections in the 

## Searching Elastic Search

In [31]:
from elasticsearch import Elasticsearch

In [32]:
es_client = Elasticsearch("http://localhost:9200")

In [33]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [35]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [36]:
from tqdm.auto import tqdm

In [37]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:08<00:00, 105.65it/s]


In [38]:
query = 'I just discovered the course. Can I still join it?'

In [39]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [40]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [41]:
rag(query)

"Yes, you can still join the course even if it has already started. You are still eligible to submit homeworks, but be aware that there will be deadlines for turning in the final projects. If you want to follow the course at your own pace, you can do so after it finishes, as all materials will be kept. You can also start installing and setting up the dependencies and requirements, and look over the prerequisites and syllabus before starting the course. Additionally, you can get support in the Slack channel even if you take the course in the self-paced mode. You don't need a confirmation email after registering for the course, you can just start learning and submitting homework without registering."

In [42]:
query = "how do I run kafka?"

In [43]:
rag(query)

'To run Kafka, you need to follow the instructions provided in the "Module 6: streaming with kafka" section of the FAQ database.\n\nFirst, you need to create a virtual environment and install the required packages. Here are the steps:\n\n1. Create a virtual environment:\n   ```\n   python -m venv env\n   ```\n2. Activate the virtual environment:\n   ```\n   source env/bin/activate\n   ```\n   (Note: For Windows, use `env/Scripts/activate` instead of `env/bin/activate`)\n3. Install the required packages:\n   ```\n   pip install -r ../requirements.txt\n   ```\n\nNow, you can run the Kafka producer and consumer using the command provided in the FAQ database. For example, to run the producer, you can use the following command:\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\nNote that you need to replace `<jar_name>` with the actual name of the JAR file in your project directory.\n\nSimilarly, you can follow the instructions in